# Character-level RNN with categorization and SMILES string encoding for molecular formulas

## Problem Definition

I am implementing and analyzing the performance of a recurrent neural network that predicts the most likely letters to continue a given string by category. So, for example, if we have a string "Mid", the model trained for the English language will likely predict the following letter to be 'd' to get closer to the final word 'middle.' However, if a given the word is said to be Estonian, the next predicted letter will likely be 'a' to complete the word 'Mida,' which translates to 'What' in English. 

I took the PyTorch tutorial "NLP from scratch: Generating names with a Character-Level RNN" as the basis for my algorithm. The PyTorch library is more easily understood and debugged by Python developers for its syntax than Tensorflow. Although TensorFlow is better for high-performance models and large datasets, Python performance in PyTorch is generally a bit faster (Abdullahi, 2022.) That makes this implementation of RNN an interesting case study.

In Figure 1, you can see the diagram for this RNN. 


From this diagram, you can see the five layers of the network (blue and green boxes):

<img src="Figure1.jpg" alt="Figure 1" width="380" height="500">

- i2o: input to output
Finds the optimized features matrix for the following current letter.

- i2h: input to hidden
Updates hidden state based on the current new character.

- o2o: output to output
Processes current output once again with the updated hidden state.

- Dropout:
Turns random parts of the input 2d matrix into zeroes to add noise.

- Softmax:
Predicts Multinomial Probability distributions.

## Deep Dive into the structure of the RNN

It runs every letter through the network one by one and calculates the output features for the letters that are most likely to come next, yet as a 2d matrix with the one-hot vector encodings in the input-to-output ( i2o ) layer. This matrix will later be converted into a probability distribution that would directly tell us the most probable letters to come. 

The algorithm also keeps gradients that optimize these features to the expected ones as hidden states in the input-to-hidden ( i2h ) layer. It combines the last hidden state with the information about the current letter present to predict the features of the following letter.

However, it also accounts for the word category as it processes the input for each network unroll, making this network a bit more complex and advanced. Instead of, for example, creating three different networks for the three different categories we want to describe, the algorithm accounts for the category by concatenating the tensors for it, and the current letter (aka one-hot vector encodings of them) with the hidden state from the previous unroll at every run for the letters in a string. This is performed at the ‘combined’ box in the diagram above (Figure 1.)

The input-to-output ( i2o ) layer finds the likely features (representative vectors) for the letters to come after the current one, as mentioned above. However, we pick the current next letter based only on the very last optimized features matrix. If we need to predict more than one letter, we treat this new letter as a part of the input string and reoptimize vectors for as many letters as required following the same process.

Now, let’s look at the hidden layer again. For the first unrolling of the network, aka the first letter, it is just a vector of zeroes with a length equal to the number of neurons we want to have in the hidden state (initialized using torch.zeros(1, self.hidden_size) in PyTorch.) Then with every letter analyzed, the hidden state is updated to account for the previous letters. It is achieved in the input-to-hidden layer using a linear activation function.

The output-to-output ( o2o ) layer just allows us to run the features optimization process once again on the current output and update the hidden state to improve our predictions at a current letter. The dropout layer zeroes random parts of the input 2d o2o matrix to simulate the corruption of the file and add noise and, therefore, help us avoid overfitting.

Finally, the Softmax layer predicts the final Multinomial Probability distributions, aka probabilities of the output letters that would likely complete the input string. We pick the one that is most likely according to this distribution.

## Dataset

I decided to experiment with the string encoding of molecular formulas called SMILES because I like chemistry. SMILES recognizes all the atoms in the periodic table and represents them using a respective atomic symbol (e.g., Helium would be just 'He'.) Aromatic atoms are lowercase. Different bonds are defined using '-', '=', '#', '*', '.' and different branches are grouped within parentheses (SMILES Tutorial | Research | US EPA, n.d.) In Figure 2 below, you can see some molecules and their corresponding SMILES encodings as an example.

<img src="Figure2.jpg" alt="Figure 2" width="200" height="100">

This encoding can be used to analyze chemicals using data science because generated strings represent the chemical structure of the molecules. This structure can then be used to predict qualitative or quantitative characteristics of different molecules, such as acidity, and types of organic compounds, such as alkanes, alkenes, and alkyne. In this assignment, I focused on just predicting the most likely atoms to follow in the input molecule. For example, we could predict 'C' in the 'CCC' input string to complete a butane SMILES formula.

I used a dataset from the Amsterdam University of Applied Sciences website, which contains 31,722 molecules and their SMILES and some other characteristics, such as monoisotopic masses and lethal doses (University of Amsterdam / Amsterdam University of Applied Sciences, 2022.)

![Figure3.jpg](Figure3.jpg)

I am going to divide this dataset into three categories based on the lethal doses of the molecules (<3 (-Log(mol/L)), 3-6 (-Log(mol/L)), >6 (-Log(mol/L)) so that I can predict atoms that are most likely to continue the molecule of a given approximate toxicity level.

## Solution Specification

In Figure 4, you can see the output for the starting ‘C’ and ‘N’ letters which stand for Carbon and Nitrogen for the toxicity under 3 (-Log(mol/L).) The predicted atoms are very similar regardless of the toxicity category, potentially implying that a set of atoms is insufficient to infer a toxicity level. Likely it also depends on the arrangement of these atoms.

<img src="Figure4.jpg" alt="Figure 4" width="200" height="100">

As expected, most predicted atoms are Carbons because 78% of the first dataset consists of Carbon atoms. I calculated that value by counting all Carbons in the first category and dividing them by the total sum of all the strings in the category <img src="Figure5.jpg" alt="Figure 5" width="300" height="150">

At the same time, the model was able to learn that the molecule rarely has just Carbons. So, it added some variation by introducing Oxygen and Nitrogen, the following most common atoms in organic compounds, to the string as well (Introduction to Organic Chemistry, n.d.) That means the model recognizes the most likely atoms to fill the molecules from the pure probabilities and statistics perspective. 

However, it doesn't learn anything about the feasible molecules. Even though it is possible to get a molecule with 19 Carbons and 2 Oxygens (examples are Isopropyl palmitate, Pristanic acid, and Nonadecanoic acid), it doesn't structure them correctly (Wikipedia contributors, 2021.) It doesn't see that there must be at least one carbon ring so the molecule holds together (Figures 6-8.)

<img src="Figures6-8.jpg" alt="Figure 6-8" width="600" height="100">

At the same time, it suggests reasonable baseline sets of atoms that can be rearranged to get a valid molecule. Therefore, this model can be a creative statistical tool for brainstorming new chemical structures and an interesting machine-learning case study that helps us better understand how RNNs view data.

## Testing and Analysis 

I will look at log loss to see how far the predictions of most likely ‘next-to-go’ letters are from the actual probabilities. Log loss metric accompanies Softmax activation function and calculates how far the probabilities of given letters being the next ones for a given string in a given category are from the true ones. The lower the log loss the better because that implies that our prediction is close to accurate. In the Figure 9 below you can see the Log Loss plot for the PyTorch Character-level RNN used for learning the SMILES dataset. Based on this figure the model learns really fast. In around 20 iterations, the network converges to a relatively stable state of around 1.575 log loss. But also, it goes to as low as 1.4 log loss.

<img src="Figures9-10.jpg" alt="Figures 9-10" width="550" height="200">

Also, 1.4 log loss implies a pretty small distance from the predictions to the true values. Therefore, we not just learn fast but also learn to predict very accurately. If we compare the language word completion for different languages in the PyTorch tutorial, the log loss for this RNN gets to the min of around 2.25, which is higher than the 1.4 log loss we have for SMILES (Figure 10.) This can happen because natural language words rarely consist of only one or three letters → are more diverse →
harder to predict correctly. 

However, we should remember that this metric penalizes minority classes a lot; therefore, majority classes, such as letters ‘C’, ‘N,’ and ‘O’ in our case, can be unreasonably prioritized. So, it reduces the chance of getting more diverse molecules, which might not be suitable if our purpose is trying to get as many very different molecules as possible.

##References 

- Abdullahi, A. (2022, July 26). PyTorch vs TensorFlow: Deep Learning Framework Comparison.
CIO Insight. https://www.cioinsight.com/big-data/pytorch-vs-tensorflow/

- Introduction to Organic Chemistry. (n.d.).
https://www2.chemistry.msu.edu/faculty/reusch/virttxtjml/chapt1.htm

- NLP From Scratch: Generating Names with a Character-Level RNN — PyTorch Tutorials
1.13.0+cu117 documentation. (n.d.).
https://pytorch.org/tutorials/intermediate/char_rnn_generation_tutorial.html

- SMILES Tutorial | Research | US EPA. (n.d.).
https://archive.epa.gov/med/med_archive_03/web/html/smiles.html

- University of Amsterdam / Amsterdam University of Applied Sciences. (2022, June 17). Norman
descriptors and predicted LC50. Figshare.
https://uvaauas.figshare.com/articles/dataset/Norman_descriptors_and_predicted_LC50/20089787

- What is the most basic explanation of log loss? (n.d.). Quora. https://www.quora.com/What-is-themost-basic-explanation-of-log-loss

- Wikipedia contributors. (2020, June 3). Pristanic acid. Wikipedia.
https://en.wikipedia.org/wiki/Pristanic_acid

- Wikipedia contributors. (2021a, November 8). Isopropyl palmitate. Wikipedia.
https://en.wikipedia.org/wiki/Isopropyl_palmitate

- Wikipedia contributors. (2021b, December 13). Nonadecylic acid. Wikipedia.
https://en.wikipedia.org/wiki/Nonadecylic_acid
